In [1]:
!pip install scipy --upgrade --quiet
import scipy
print("SciPy version:", scipy.__version__)


SciPy version: 1.15.3


In [2]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Model
from tensorflow.keras import layers
from tensorflow.keras.applications import MobileNetV2  # Smaller than EfficientNet
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from huggingface_hub import HfApi
from dotenv import load_dotenv
import os

print("TensorFlow:", tf.__version__)

c:\Users\donof\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


TensorFlow: 2.20.0


In [3]:
load_dotenv()

True

In [4]:
api = HfApi()

In [ ]:
IMG_SIZE = 180

datagen_train = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True,
    zoom_range=0.2
)
datagen_test = ImageDataGenerator(rescale=1./255)

train_set = datagen_train.flow_from_directory(
    "../dataset/pokemon/train",
    target_size=(IMG_SIZE, IMG_SIZE),
    class_mode='categorical',
    batch_size=32
)

test_set = datagen_test.flow_from_directory(
    "../dataset/pokemon/validation",
    target_size=(IMG_SIZE, IMG_SIZE),
    class_mode='categorical',
    batch_size=32
)

print("Train samples:", train_set.samples)
print("Val samples:", test_set.samples)
print("Classes:", len(train_set.class_indices))


In [ ]:
base_model = MobileNetV2(
    weights='imagenet',
    include_top=False,
    input_shape=(IMG_SIZE, IMG_SIZE, 3)
)
base_model.trainable = False

inputs = layers.Input(shape=(IMG_SIZE, IMG_SIZE, 3))
x = base_model(inputs, training=False)
x = layers.GlobalAveragePooling2D()(x)
x = layers.Dropout(0.3)(x)
x = layers.Dense(128, activation='relu')(x)
x = layers.Dropout(0.3)(x)
outputs = layers.Dense(len(train_set.class_indices), activation='softmax')(x)

model = Model(inputs, outputs)
model.summary()


In [ ]:
callbacks = [
    EarlyStopping(monitor='val_accuracy', patience=8, restore_best_weights=True, verbose=1),
    ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=4, min_lr=1e-7, verbose=1)
]


In [ ]:
model.compile(optimizer=Adam(0.001), loss='categorical_crossentropy', metrics=['accuracy'])

history = model.fit(
    train_set,
    validation_data=test_set,
    epochs=20,
    callbacks=callbacks
)


In [ ]:
# Best validation accuracy during training
best_val_acc = max(history.history['val_accuracy'])
print("Best val_accuracy during training:", best_val_acc)

# Final evaluation on validation set
val_loss, val_acc = model.evaluate(test_set)
print("Final validation loss:", val_loss)
print("Final validation accuracy:", val_acc)

# Also print training accuracy for comparison
best_train_acc = max(history.history['accuracy'])
print("Best training accuracy:", best_train_acc)


In [ ]:
# Load the saved Keras model (or use `model` directly if it's still in memory)

# Create converter from Keras model
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

# Save the TFLite model to disk
with open("pokemon_classifier.tflite", "wb") as f:
    f.write(tflite_model)

In [5]:
api.upload_file(
    path_or_fileobj="./pokemon_mobilenetv2.tflite",
    path_in_repo="pokemon_mobilenetv2.tflite",
    repo_id=os.getenv("POKE_REPO"),
    revision="main",
    create_pr=True,
    repo_type="model"
)

HfHubHTTPError: (Request ID: Root=1-69374249-32d326774339de997ad3e438;ef5c1291-dbfe-4fe1-8bc2-7086ff967384)

403 Forbidden: Authorization error..
Cannot access content at: https://huggingface.co/api/models/daksheshgandhe/pokemon_mobilenetv2.keras/preupload/main?create_pr=1.
Make sure your token has the correct permissions.